In [2]:
#==========================#
# Import relevant packages #
#==========================#


import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import erfinv
from iminuit import Minuit
from scipy.stats import chi2
from scipy.stats import binom

### Functions

In [ ]:
#=======================================================#
# Create a function which calculates the time of flight #
#=======================================================#


def calcToF(L, c, m, p):
    return ( (L/c)*np.sqrt(1+((m*c)/p)**2) )

### Classes

In [ ]:
#============================#
# Create the Gaussian class  #
#============================#


class Gaussian(object):

    #————————————————————————————————————————————————————————#
    # Class constructor | initialize an attribute mass array #
    #————————————————————————————————————————————————————————#

    def __init__(self, mean, sigma, loBound, highBound):
   
        self.loBound = loBound
        self.highBound = highBound
        self.mean = mean
        self.sigma = sigma
        self.loBound_distribution = self.mean - 10.*self.sigma # 10 sigma from the mean, so the range covers the whole distribution
        self.hiBound_distribution = self.mean + 10.*self.sigma # 10 sigma from the mean, so the range covers the whole distribution

        #initialize an mass list used to store mass values
        self.mass = []
    
    
    #——————————————————————————————————#
    # Evaluation method (un-normalize) #
    #——————————————————————————————————#

    def evaluate(self, x):
        return (1/(self.sigma*np.sqrt(2.*np.pi))) * np.exp( -(x-self.mean)**2/(2.*self.sigma**2) )
    

    #————————————————————————————————————————————————#
    # Evaluate the integral of the pdf within limits #
    #————————————————————————————————————————————————#

    def integrateBelow(self, loBound=loBound_distribution, highBound):
        integral_result, integral_err = quad(self.evaluate, loBound, highBound)
        return integral_result
    
    def integrateAbove(self, loBound, highBound=hiBound_distribution):
        integral_result, integral_err = quad(self.evaluate, loBound, highBound)
        return integral_result

    #————————————————————————————————————————————————————————————————————#
    # Function that generate a single random vars using np.random.normal #
    # method generated var is returned and appended to the mass list     #
    #————————————————————————————————————————————————————————————————————#

    def next(self):
        x = np.random.normal(self.mean, self.sigma, size=1).item()
        self.mass.append(x)
        return x

In [ ]:
#===========================================================================================================================#
# Create the ROC class used to calculates (alpha_i, beta_i) pairs of ToF performance for distinguishing b/w pions and kaons #
#===========================================================================================================================#


class ROC(object):


    #——————————————————————————#
    # Define class constructor #
    #——————————————————————————#

    def __init__(self, p, sigma, kaon_mass, pion_mass, scint_counters_dist, c):
        self.ToF_kaon = calcToF(scint_counters_dist, c, kaon_mass, p)
        self.ToF_pion = calcToF(scint_counters_dist, c, pion_mass, p)
        

### Code Execution (Problem Set CP5)

Problem 1

In [7]:
def run_Q1():

    # Define the relevant parameters
    claimedFaultyRate = 0.05
    sampleSize = 100
    trueFaultyRate = 0.15
    tCut = 9                # max no. of faulty monitors willing to have
    t = np.arange(0, 100, 1) # for sample of 100 monitors
    
    # Question (a) of Problem 1
    # first define binom distribution for the claimed faulty
    claimedFaultyDistribution = binom(sampleSize, claimedFaultyRate)
    
    # calculate Type-I error
    typeIMaskT = (t<=tCut)
    typeIErr = claimedFaultyDistribution.pmf(t)[typeIMaskT].sum()
    print('The type-I error is %.4f' % typeIErr)
          
    # Question (b) of Problem 1
    # first define binom distribution for the claimed faulty
    trueFaultyDistribution = binom(sampleSize, trueFaultyRate)

    # calculate Type-II error
    typeIIMaskT = (t>=tCut)
    typeIIErr = trueFaultyDistribution.pmf(t)[typeIIMaskT].sum()
    print('The type-II error is %.4f' % typeIIErr)

In [8]:
run_Q1()

The type-I error is 0.9718
The type-II error is 0.9725


Problem 2

In [ ]:
def run_Q2():
    
    # Define relevant parameters
    k_mass = 493.7 # MeV/c^2
    pi_mass = 139.6 # MeV/c^2
    scintCountersDist = 20 # m
    c = 3.0e8 #m/s
    timeResMean = 0.4 #ns
